In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

 Activating environment at `~/Dropbox/class/M1399.000200/2020/Project.toml`


Status `~/Dropbox/class/M1399.000200/2020/Project.toml`
  [6e4b80f9] BenchmarkTools v0.5.0
  [31c24e10] Distributions v0.23.10
  [c91e804a] Gadfly v1.3.0
  [bd48cda9] GraphRecipes v0.5.4
  [47be7bcc] ORCA v0.5.0
  [a03496cd] PlotlyBase v0.4.1
  [f0f68f2c] PlotlyJS v0.14.0
  [91a5bcdd] Plots v1.6.1
  [438e738f] PyCall v1.91.4
  [d330b81b] PyPlot v2.9.0
  [6f49c342] RCall v0.13.7
  [8f399da3] Libdl 
  [10745b16] Statistics 


# Gaussian Elimination and LU Decomposition

* Goal: solve linear equation
$$
\mathbf{A} \mathbf{x} = \mathbf{b}.
$$
For simplicity we consider a square matrix $\mathbf{A} \in \mathbb{R}^{n \times n}$.

* [History](https://en.wikipedia.org/wiki/Gaussian_elimination#History): the method is named after Carl Friedrich Gauss (1777–1855), although it stems from the notes of Isaac Newton. If fact, GE was known to [Chinese mathematicians](https://en.wikipedia.org/wiki/The_Nine_Chapters_on_the_Mathematical_Art) as early as 179 AD.


* A [toy example](./gelu.pdf).

In [3]:
A = [2.0 -4.0 2.0; 4.0 -9.0 7.0; 2.0 1.0 3.0]

3×3 Array{Float64,2}:
 2.0  -4.0  2.0
 4.0  -9.0  7.0
 2.0   1.0  3.0

In [4]:
b = [6.0, 20.0, 14.0]

3-element Array{Float64,1}:
  6.0
 20.0
 14.0

In [5]:
# Julia way to solve linear equation
# equivalent to `solve(A, b)` in R
A \ b

3-element Array{Float64,1}:
 2.000000000000001
 1.0
 2.9999999999999996

What happens when we call `A \ b` to solve a linear equation?

## Elementary operator matrix

* **Elementary operator matrix** is the identity matrix with the 0 in position $(j,k)$ replaced by $c$
$$
    \mathbf{E}_{jk}(c) = \begin{pmatrix}
    1 & & \\
    & \ddots & \\
    & & 1 & \\
    & & & \ddots & \\
    & & c & & 1 & \\
    & & & & & \ddots \\
    & & & & & & 1
    \end{pmatrix} = \mathbf{I} + c \mathbf{e}_j \mathbf{e}_k^T.
$$

* $\mathbf{E}_{jk}(c)$ is unit triangular, full rank, and its inverse is
$$
    \mathbf{E}_{jk}^{-1}(c) = \mathbf{E}_{jk}(-c).
$$

* $\mathbf{E}_{jk}(c)$ left-multiplies an $n \times m$ matrix $\mathbf{X}$ effectively replace its $j$-th row $\mathbf{x}_{j\cdot}$ by $c \mathbf{x}_{k \cdot} + \mathbf{x}_{j \cdot}$
$$
    \mathbf{E}_{jk}(c) \times \mathbf{X} = \mathbf{E}_{jk}(c) \times \begin{pmatrix}
    & & \\
    \cdots & \mathbf{x}_{k\cdot} & \cdots \\
    & & \\
    \cdots & \mathbf{x}_{j\cdot} & \cdots \\
    & & 
    \end{pmatrix} = \begin{pmatrix}
    & & \\
    \cdots & \mathbf{x}_{k\cdot} & \cdots \\
    & & \\
    \cdots & c \mathbf{x}_{k\cdot} + \mathbf{x}_{j\cdot} & \cdots \\
    & & 
    \end{pmatrix}.
$$
$2m$ flops (why?).

* Gaussian elimination applies a sequence of elementary operator matrices to transform the linear system $\mathbf{A} \mathbf{x} = \mathbf{b}$ to an upper triangular system
$$
\begin{eqnarray*}
    \mathbf{E}_{n,n-1}(c_{n,n-1}) \cdots \mathbf{E}_{21}(c_{21}) \mathbf{A} \mathbf{x} &=& \mathbf{E}_{n,n-1}(c_{n,n-1}) \cdots \mathbf{E}_{21}(c_{21}) \mathbf{b} \\
    \mathbf{U} \mathbf{x} &=& \mathbf{b}_{\text{new}}.
\end{eqnarray*}
$$
    
    Column 1:

In [ ]:
E21 = [1.0 0.0 0.0; -2.0 1.0 0.0; 0.0 0.0 1.0]

In [ ]:
# zero (2, 1) entry
E21 * A   # Step 1, A'

In [ ]:
E31 = [1.0 0.0 0.0; 0.0 1.0 0.0; -1.0 0.0 1.0]

In [ ]:
# zero (3, 1) entry
E31 * E21 * A  # Step 2, A''

    Column 2:

In [ ]:
E32 = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 5.0 1.0]

In [ ]:
# zero (3, 2) entry
E32 * E31 * E21 * A   # Step 3, A'''

## Gauss transformations

* For the first column,
$$
    \mathbf{M}_1 = \mathbf{E}_{n1}(c_{n1}) \cdots \mathbf{E}_{31}(c_{31}) \mathbf{E}_{21}(c_{21}) = \begin{pmatrix}
    1 & \\
    c_{21} & \\
    & \ddots & \\
    c_{n1} & & 1
    \end{pmatrix}
$$  
For the $k$-th column,
$$
	\mathbf{M}_k = \mathbf{E}_{nk}(c_{nk}) \cdots \mathbf{E}_{k+1,k}(c_{k+1,k}) = \begin{pmatrix}
	1 & \\
	& \ddots \\
	& & 1 & \\
	& & c_{k+1,k} & 1\\
	& & \vdots & & \ddots \\
	& &  c_{n,k} & & & 1
	\end{pmatrix}.
$$

* $\mathbf{M}_1, \ldots, \mathbf{M}_{n-1}$ are called the **Gauss transformations**.

In [ ]:
M1 = E31 * E21   # inv(L2 * L1)

In [ ]:
M2 = E32    # inv(L3)

* Gauss transformations $\mathbf{M}_k$ are unit triangular, full rank, with inverse
$$
	\mathbf{M}_k^{-1} = \mathbf{E}_{k+1,k}^{-1}(c_{k+1,k}) \cdots \mathbf{E}_{nk}^{-1}(c_{nk}) = \begin{pmatrix}
	1 & \\
	& \ddots \\
	& & 1 & \\
	& & - c_{k+1,k}\\
	& & \vdots & & \ddots \\
	& & - c_{n,k} & & & 1
	\end{pmatrix}.
$$

In [ ]:
inv(M1)    # L2 * L1. inv(A) give the inverse of A, but use with caution (see below)

In [ ]:
inv(M2)    # L3

## LU decomposition

Gaussian elimination does
$$
\mathbf{M}_{n-1} \cdots \mathbf{M}_1 \mathbf{A} = \mathbf{U}.
$$  
Let

\begin{equation*}
\mathbf{L} = \mathbf{M}_1^{-1} \cdots \mathbf{M}_{n-1}^{-1} = \begin{pmatrix}  
	1 & & & & \\
	- c_{21} & \ddots & & & \\
	& & 1 & & \\
	- c_{k+1,1} & & - c_{k+1,k} & & \\
	& & \vdots & & \ddots \\
	- c_{n1} & & - c_{nk} & & & 1
	\end{pmatrix}.
\end{equation*}
Thus we have the **LU decomposition**
$$
\mathbf{A} = \mathbf{L} \mathbf{U},
$$
where $\mathbf{L}$ is **unit** lower triangular and $\mathbf{U}$ is upper triangular.

* LU decomposition exists if the principal sub-matrix $\mathbf{A}[1:k, 1:k]$ is non-singular for $k=1,\ldots,n-1$.

* If the LU decomposition exists and $\mathbf{A}$ is non-singular, then the LU decmpositon is unique and
$$
    \det(\mathbf{A}) = \det(\mathbf{L}) \det(\mathbf{U}) = \prod_{k=1}^n u_{kk}.
$$
    - This is basically how Julia, R, and MATLAB compute determinants.

* The whole LU algorithm is done *in place*, i.e., $\mathbf{A}$ is **overwritten** by $\mathbf{L}$ and $\mathbf{U}$.
```Julia
for k=1:n-1
    for i=k+1:n
        A[i, k] /= A[k, k]
        for j=k+1:n
            A[i, j] -= A[i, k] * A[k, j]
        end
    end
end
```

* The LU decomposition costs
$$
    2(n-1)^2 + 2(n-2)^2 + \cdots + 2 \cdot 1^2 \approx \frac 23 n^3 \quad \text{flops}.
$$

<img src="http://www.netlib.org/utk/papers/factor/_25826_figure159.gif" width="500" align="center"/>

Source: <http://www.netlib.org>

* Actual implementations can differ: outer product LU ($kij$ loop), block outer product LU (higher level-3 fraction), Crout's algorithm ($jki$ loop).

* Given LU decomposition $\mathbf{A} = \mathbf{L} \mathbf{U}$, solving $(\mathbf{L} \mathbf{U}) \mathbf{x} = \mathbf{b}$ for one right hand side costs $2n^2$ flops:
    - One forward substitution ($n^2$ flops) to solve
    $$
    \mathbf{L} \mathbf{y} = \mathbf{b}
    $$
    - One back substitution ($n^2$ flops) to solve
    $$
    \mathbf{U} \mathbf{x} = \mathbf{y}
    $$
    
* Therefore to solve $\mathbf{A} \mathbf{x} = \mathbf{b}$ via LU, we need a total of
$$
    \frac 23 n^3 + 2n^2 \quad \text{flops}.
$$

* If there are multiple right hand sides, LU only needs to be done once.

## Matrix inversion

* For matrix inversion, there are $n$ right hand sides $\mathbf{e}_1, \ldots, \mathbf{e}_n$. Taking advantage of 0s reduces $2n^3$ flops to $\frac 43 n^3$ flops. So **matrix inversion via LU** costs
$$
\frac 23 n^3 + \frac 43 n^3 = 2n^3 \quad \text{flops}.
$$

* **No inversion mentality**:  
    > **Whenever we see matrix inverse, we should think in terms of solving linear equations.**
    > In short, no ```inv(A)```.

    We do not compute matrix inverse unless  
    1. it is necessary to compute standard errors  
    2. number of right hand sides is much larger than $n$  
    3. $n$ is small
 

   
## Pivoting    

* Let
$$
    \mathbf{A} = \begin{bmatrix}
    0 & 1 \\
    1 & 0 \\
    \end{bmatrix}.
$$
Is there a solution to $\mathbf{A} \mathbf{x} = \mathbf{b}$ for an arbitrary $\mathbf{b}$? Does GE/LU work for $\mathbf{A}$?

* What if, during LU procedure, the **pivot** $a_{kk}$ is 0 or nearly 0 due to underflow?  
     **Example**:
$$
\begin{split}
0.001x_1 + x_2 &= 1  \\
     x_1 + x_2 &= 2 
\end{split}
$$
with solution

$$
\begin{bmatrix} x_1 \\ x_ 2 \end{bmatrix}
=
\begin{bmatrix} 1.001001 \\ 0.998999 \end{bmatrix}
\approx
\begin{bmatrix} 1.0 \\ 1.0 \end{bmatrix}
.
$$

* With two significant digits, GE yields 
$$
\begin{split}
0.001x_1 + x_2 &= 1  \\
     (1-1000)x_2 &= 2 - 1000 
\end{split}
$$
or
$$
\begin{split}
0.001x_1 + x_2 &= 1 \\
     -1000 x_2 &= -1000 
\end{split}
,
$$
yielding
$$
\begin{bmatrix} x_1 \\ x_ 2 \end{bmatrix}
=
\begin{bmatrix} 0.0 \\ 1.0 \end{bmatrix}
!
$$


* Solution: pivoting.

$$
\begin{split}
     x_1 + x_2 &= 2  \\
0.001x_1 + x_2 &= 1  
\end{split}
$$

* With two significant digits, GE yields 

$$
\begin{split}
     x_1 + x_2 &= 2  \\
     (1-0.001)x_2 &= 1 - 0.001 
\end{split}
$$

or

$$
\begin{split}
     x_1 + x_2 &= 2  \\
     1.0 x_2 &= 1.0
\end{split}
,
$$

yielding

$$
\begin{bmatrix} x_1 \\ x_ 2 \end{bmatrix}
=
\begin{bmatrix} 1.0 \\ 1.0 \end{bmatrix}
.
$$


* **Partial pivoting**: before zeroing the $k$-th column, move the row with $\max_{i=k}^n |a_{ik}|$ is into the $k$-th row (called GEPP).

* LU with partial pivoting yields
$$
	\mathbf{P} \mathbf{A} = \mathbf{L} \mathbf{U},
$$
where $\mathbf{P}$ is a permutation matrix, $\mathbf{L}$ is unit lower triangular with $|\ell_{ij}| \le 1$, and $\mathbf{U}$ is upper triangular.

* Complete pivoting (GECP): Do both row and column interchanges so that the largest entry in the submatrix `A[k:n, k:n]` is permuted to the $(k,k)$-th entry. This yields the decomposition 
$$
\mathbf{P} \mathbf{A} \mathbf{Q} = \mathbf{L} \mathbf{U},
$$
where $|\ell_{ij}| \le 1$.

* GEPP is the most commonly used methods for solving **general** linear systems. GECP is more stable but costs more computation. Partial pivoting is stable most of times (which is partially because GEPP guarantees $|\ell_{ij}| \le 1$).

## Implementation

* LAPACK: [`?GETRF`](http://www.netlib.org/lapack/explore-html/dd/d9a/group__double_g_ecomputational_ga0019443faea08275ca60a734d0593e60.html#ga0019443faea08275ca60a734d0593e60) does $\mathbf{P} \mathbf{A} = \mathbf{L} \mathbf{U}$ (LU decomposition with partial pivoting) in place.

* R: `solve()` implicitly performs LU decomposition (wrapper of LAPACK routine `DGESV`). `solve()` allows specifying a single or multiple right hand sides. If none, it computes the matrix inverse. The `matrix` package contains `lu()` function that outputs `L`, `U`, and `pivot`.

* Julia: 
    - [`LinearAlgebra.lu`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.lu).
    - [`LinearAlgebra.lu!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.lu!). In-place version. Input matrix gets overwritten with L and U.
    - Or call LAPACK wrapper function [`LAPACK.getrf!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.getrf!) directly.
    - Other LU-related LAPACK wrapper functions: [`getrs`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.getrs!), [`gesv`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.gesv!), [`gesvx`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.gesvx!), [`trtri`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.trtri!) (inverse of triangular matrix), [`trtrs`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LAPACK.trtrs!).

In [ ]:
A

In [ ]:
using LinearAlgebra

# second argument indicates partial pivoting (default) or not
alu = lu(A)
typeof(alu)

In [ ]:
fieldnames(typeof(alu))

In [ ]:
alu.L

In [ ]:
alu.U

In [ ]:
alu.p

In [ ]:
alu.P

In [ ]:
alu.L * alu.U

In [ ]:
A[alu.p, :]

In [ ]:
# this is doing two triangular solves, 2n^2 flops
alu \ b

In [ ]:
det(A) # this does LU!

In [ ]:
det(alu) # this is cheap

In [ ]:
inv(A) # this does LU!

In [ ]:
inv(alu) # this is cheap

## Further reading

* Sections II.5.2 and II.5.3, [Computational Statistics](https://link.springer.com/book/10.1007%2F978-0-387-98144-4) by James Gentle (2010).

* Chapter 2, [Applied Numerical Linear Algebra](https://doi.org/10.1137/1.9781611971446) by James W. Demmel (1997).

* Chapter 3, [Matrix Computation](https://www.amazon.com/Computations-Hopkins-Studies-Mathematical-Sciences/dp/1421407949/ref=sr_1_1?keywords=matrix+computation+golub&qid=1567157884&s=gateway&sr=8-1) by Gene Golub and Charles Van Loan (2013).

<img src="https://images-na.ssl-images-amazon.com/images/I/41f5vxegABL._SY344_BO1,204,203,200_.jpg" width="250" align="center"/>

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.